测试内外圆检测

In [12]:
import numpy as np
import cv2
from util.innerCircle import innerCircle
from util.outerCircle import outerCircle
from util.normalize import normalize
from util.feature import swtFeatureMap, mallatFeatureMap
from util.visualization import displayCircle

img_path = './photo/dz/L/1.jpeg'
img = cv2.imread(img_path, 0)
inner = innerCircle(img)
outer = outerCircle(img, inner)
displayCircle(img, outer[0], outer[1], outer[2], inner[0], inner[1], inner[2])

任选两张图像进行相似度比对

In [4]:
import cv2
import numpy as np
from util.feature import getFeatureMap

feature_1 = getFeatureMap(cv2.imread("./photo/pc/L/1.jpeg", 0), 'swt')
feature_2 = getFeatureMap(cv2.imread("./photo/pc/L/15.jpeg", 0), 'swt')
feature_3 = getFeatureMap(cv2.imread("./photo/lxy/R/10.jpeg", 0), 'swt')
feature_4 = getFeatureMap(cv2.imread("./photo/lxy/R/21.jpeg", 0), 'swt')
same_distance = np.count_nonzero(feature_1 != feature_2)
diff_distance = np.count_nonzero(feature_3 != feature_4)
print(same_distance)
print(diff_distance)
# cv2.imwrite('./1.bmp', feature_1)
# cv2.imread('./1.bmp', -1)

60432
50395


array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 1, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       [1, 0, 1, ..., 1, 0, 0]], dtype=uint8)

对所有图像提取特征

In [1]:
import os
import cv2
from util.feature import getFeatureMap

feature_path = r'.\feature'  # 特征目录
dataset_path = r'.\photo'
i = 0
for path, dir_list, file_list in os.walk(dataset_path):
    # print(path)
    # print(dir_list)
    # print(file_list)
    save_dir = feature_path + path[len(dataset_path):]
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for file_name in file_list:
        img_path = os.path.join(path, file_name)
        save_path = feature_path + img_path[len(dataset_path):1 - len('.\jpeg')] + '.bmp'
        # print(save_path)
        feature = getFeatureMap(cv2.imread(img_path, 0), 'swt')
        cv2.imwrite(save_path, feature)
        i += 1

任选一张图像判断与其最相似的虹膜所属者以及左右眼

In [28]:
%%time

persons = 4
photos_per_eye = 20
score_dict = {}
test_img_path = './photo/dz/R/1.jpeg'
test_img = cv2.imread(test_img_path, 0)
test_img_feature = getFeatureMap(test_img, 'swt')

score_per_eye = 0
feature_path = r'.\feature'  # 记录初始路径
for path, dir_list, file_list in os.walk(feature_path):
    # print(path)
    # print(dir_list)
    # print(file_list)
    if len(file_list) != 0:
        for img_path in file_list:
            img_feature = cv2.imread(os.path.join(path, img_path), -1)
            distance = np.count_nonzero(test_img_feature != img_feature)
            # print(distance)
            score_per_eye += distance
        path_split = path.split('\\')
        score_dict[path_split[-2] + '-' + path_split[-1]] = score_per_eye / len(file_list)
        # print(score_per_eye / len(file_list))
        score_per_eye = 0

print(score_dict)
sorted_list = sorted(score_dict.items(), key=lambda x: x[1], reverse=False)
print(sorted_list[0][0])

{'dz-L': 67580.06666666667, 'dz-R': 55252.709677419356, 'lxy-L': 71105.33333333333, 'lxy-R': 71669.1875, 'pc-L': 69550.2, 'pc-R': 70783.79069767441, 'ycr-L': 68509.77777777778, 'ycr-R': 70652.56410256411}
dz-R
Wall time: 148 ms


使用封装好的util包

In [1]:
import cv2
from util.feature import generateFeatureDataset
from util.contrast import contrast

test_img_path = './photo/lxy/R/1.jpeg'
test_img = cv2.imread(test_img_path, 0)
# generateFeatureDataset()
name, side, scores = contrast(test_img)
print(name)
print(side)
print(scores)


lxy
R
[('lxy-R', 56951.25), ('ycr-L', 68283.38888888889), ('pc-L', 68407.24), ('ycr-R', 68951.89743589744), ('dz-L', 68986.8), ('pc-R', 69243.46511627907), ('lxy-L', 72036.08333333333), ('dz-R', 73366.96774193548)]
